In [ ]:
# Install modified lib from github
!python -m pip install --no-cache-dir -U git+https://github.com/ffflowww/optillm.git@qwen-vllm

In [2]:
from openai import OpenAI
import requests
import json
import numpy as np
from typing import List, Tuple
from optillm.cot_decoding_vllm import cot_decode_vllm, build_qwen_chat_prompt
import warnings

/home/f10w/anaconda3/envs/lookahead/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/f10w/anaconda3/envs/lookahead/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/f10w/anaconda3/envs/lookahead/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/f10w/anaconda3/envs/lookahead/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Plea

In [3]:
model = "Qwen/Qwen2.5-72B-Instruct"
base_url = ""
api_key = ""

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

## Initial promt

In [5]:
system_prompt = "You are a smart assistant."
user_prompt = "In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?"

prompt = build_qwen_chat_prompt(system_prompt, user_prompt)
prompt

'<|im_start|>system\nYou are a smart assistant.<|im_end|>\n<|im_start|>user\nIn a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?<|im_end|>\n<|im_start|>assistant\n'

## Case 1. Get the best answer

In [6]:
response = cot_decode_vllm(prompt, client, model)
response

("To determine the percentage of students enrolled in hip-hop dance, we will follow these steps:\n\n1. Calculate the number of students enrolled in contemporary dance.\n2. Determine the number of remaining students after the contemporary dance enrollment.\n3. Calculate the number of students enrolled in jazz dance from the remaining students.\n4. Determine the number of students enrolled in hip-hop dance.\n5. Convert the number of students enrolled in hip-hop dance to a percentage of the total class.\n\nLet's start with step 1:\n\n1. Calculate the number of students enrolled in contemporary dance:\n   \\[\n   \\text{Number of students in contemporary dance} = 20\\% \\text{ of } 20 = 0.20 \\times 20 = 4\n   \\]\n   So, 4 students are enrolled in contemporary dance.\n\n2. Determine the number of remaining students after the contemporary dance enrollment:\n   \\[\n   \\text{Remaining students} = 20 - 4 = 16\n   \\]\n\n3. Calculate the number of students enrolled in jazz dance from the rem

## Case 2. Show all answers and its confidence scores (is_return_all=True). Showcase of all supported parameters

In [8]:
response = cot_decode_vllm(base_prompt=prompt,
                           client=client,
                           model=model,
                           k=10,  # default
                           max_new_tokens=512,  # default
                           temperature=1.,  # default
                           top_p=1.,  # default
                           aggregate_paths=False,  # default
                           is_return_all=True
                          )

for i, resp in enumerate(response):
    print(f"({i + 1}/{len(response)}) Score: {resp[1]}")
    print(f"Answer: {resp[0]}\n")

(0/10)Score: 0.963262832144993
Answer: To determine the percentage of students enrolled in hip-hop dance, we start by calculating the number of students in each dance category step by step.

1. **Calculate the number of students enrolled in contemporary dance:**
   \[
   20\% \text{ of } 20 = 0.20 \times 20 = 4 \text{ students}
   \]

2. **Determine the number of remaining students after those enrolled in contemporary dance:**
   \[
   20 - 4 = 16 \text{ students}
   \]

3. **Calculate the number of students enrolled in jazz dance:**
   \[
   25\% \text{ of } 16 = 0.25 \times 16 = 4 \text{ students}
   \]

4. **Determine the number of students enrolled in hip-hop dance:**
   \[
   20 - 4 \text{ (contemporary)} - 4 \text{ (jazz)} = 12 \text{ students}
   \]

5. **Calculate the percentage of students enrolled in hip-hop dance:**
   \[
   \frac{12}{20} \times 100\% = 60\%
   \]

Thus, the percentage of students enrolled in hip-hop dance is \(\boxed{60\%}\).

(1/10)Score: 0.957910968416689

## Playground below (test zone)

In [ ]:
"""THIS IS AN EXACT RIGHT PROMPT BELOW FOR QWEN 2.5 INSTRUCT"""

"""
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?<|im_end|>
<|im_start|>assistant

"""

In [314]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": "Hello!"}
  ]
)

completion.choices[0].message

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', function_call=None, tool_calls=[])